In [91]:
import pandas as pd
import numpy as np

import locale
locale.setlocale(locale.LC_ALL,'')

'Spanish_Mexico.1252'

In [92]:
df = pd.read_excel("data.xlsx",'data')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2103 entries, 0 to 2102
Data columns (total 15 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Número de movimiento      2103 non-null   int64  
 1   Documento                 2103 non-null   object 
 2   Clave de artículo         2103 non-null   int64  
 3   Descripción               2103 non-null   object 
 4   Línea                     2103 non-null   object 
 5   Clave cliente/proveedor   2103 non-null   object 
 6   Concepto                  2103 non-null   int64  
 7   Fecha                     2103 non-null   object 
 8   Vendedor                  2072 non-null   float64
 9   Precio                    2103 non-null   float64
 10  Costo                     2103 non-null   float64
 11  Almacén                   2103 non-null   int64  
 12  Tipo de documento         2103 non-null   object 
 13  Costo operado             2103 non-null   object 
 14  Cantidad

In [93]:
df["Clave de artículo "] = df["Clave de artículo "].astype(object)
df["Fecha "] = pd.to_datetime(df["Fecha "], dayfirst=True)
df["Cantidad "] = pd.to_numeric(df["Cantidad "].str.replace(',',''))
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2103 entries, 0 to 2102
Data columns (total 15 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Número de movimiento      2103 non-null   int64         
 1   Documento                 2103 non-null   object        
 2   Clave de artículo         2103 non-null   object        
 3   Descripción               2103 non-null   object        
 4   Línea                     2103 non-null   object        
 5   Clave cliente/proveedor   2103 non-null   object        
 6   Concepto                  2103 non-null   int64         
 7   Fecha                     2103 non-null   datetime64[ns]
 8   Vendedor                  2072 non-null   float64       
 9   Precio                    2103 non-null   float64       
 10  Costo                     2103 non-null   float64       
 11  Almacén                   2103 non-null   int64         
 12  Tipo de documento   

In [94]:
dfA = df[df["Almacén "] == 1][["Fecha ","Cantidad "]]

dfprov = dfA.groupby("Fecha ").sum()
dfprov["Cantidad "] = dfprov[dfprov["Cantidad "] < 0 ] * -1
dfprov["Dias "] = 1
dfprov["Probabilidad "] = 1 / dfprov["Cantidad "].count()

tabla = dfprov.groupby("Cantidad ").agg({"Dias ":'count',"Probabilidad ":'sum'})
tabla["Si-dias "] = tabla["Dias "].cumsum()
tabla["No-dias "] = tabla["Dias "].sum() - tabla["Si-dias "]
tabla["Si-% "] = tabla["Si-dias "] / tabla["Dias "].sum()
tabla["No-% "] = tabla["No-dias "] / tabla["Dias "].sum()

tabla

,Dias,Probabilidad,Si-dias,No-dias,Si-%,No-%
Cantidad,,,,,,
1.0,1,0.002732,1,365,0.002732,0.997268
2.0,8,0.021858,9,357,0.024590,0.975410
3.0,7,0.019126,16,350,0.043716,0.956284
4.0,2,0.005464,18,348,0.049180,0.950820
5.0,10,0.027322,28,338,0.076503,0.923497
...,...,...,...,...,...,...
680.0,1,0.002732,362,4,0.989071,0.010929
752.0,1,0.002732,363,3,0.991803,0.008197
1005.0,1,0.002732,364,2,0.994536,0.005464


In [96]:
tabla[tabla["Si-% "] > .95].iloc[0:1]

,Dias,Probabilidad,Si-dias,No-dias,Si-%,No-%
Cantidad,,,,,,
340.0,2,0.005464,349,17,0.953552,0.046448


In [98]:
almacenes = [1,2]

writer = pd.ExcelWriter("stock-mínimo.xlsx", engine="openpyxl")
    
for almacen in almacenes:

    dfA = df[df["Almacén "] == almacen][["Fecha ","Cantidad "]]

    dfprov = dfA.groupby("Fecha ").sum()
    dfprov["Cantidad "] = dfprov[dfprov["Cantidad "] < 0 ] * -1
    dfprov["Dias "] = 1
    dfprov["Probabilidad "] = 1 / dfprov["Cantidad "].count()

    tabla = dfprov.groupby("Cantidad ").agg({"Dias ":'count',"Probabilidad ":'sum'})
    tabla["Si-dias "] = tabla["Dias "].cumsum()
    tabla["No-dias "] = tabla["Dias "].sum() - tabla["Si-dias "]
    tabla["Si-% "] = tabla["Si-dias "] / tabla["Dias "].sum()
    tabla["No-% "] = tabla["No-dias "] / tabla["Dias "].sum()
    
    tabla.to_excel(writer, sheet_name=f"A{almacen}")

writer.close()
writer.handles = None